In [ ]:
%pip install torch torchvision torchaudio
%pip install --upgrade transformers
%pip install -q -U transformers
%pip install -q bitsandbytes accelerate>=0.25.0

In [2]:
from transformers import MllamaForConditionalGeneration, AutoProcessor
from transformers import AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# quantization_config = BitsAndBytesConfig(
#     load_in_8bit=True)
# #     bnb_8bit_compute_dtype=torch.float16
# # )

model_id = "unsloth/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

# tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
!nvidia-smi

Tue Oct  8 18:46:52 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.107.02             Driver Version: 550.107.02     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:01:00.0 Off |                  Off |
|  0%   43C    P2             75W /  450W |   21056MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
# !nvidia-smi | grep 'python' | awk '{ print $5 }' | xargs -n1 kill -9

In [2]:
torch.cuda.empty_cache()

In [3]:
import pandas as pd

df = pd.read_parquet('./train-00000-of-00072.parquet')

In [14]:
# from PIL import Image
# from io import BytesIO

# # Предположим, что limited_ds[0]['image']['bytes'] содержит байты изображения
# image_bytes = df.loc[0]['image']['bytes']

# # Преобразуем байты в поток
# image_stream = BytesIO(image_bytes)

# # Используем PIL для открытия изображения
# image = Image.open(image_stream)

# # Отображение изображения
# image


In [4]:
import requests
import json

file_url = 'https://huggingface.co/datasets/huggan/wikiart/resolve/main/dataset_infos.json'
response = requests.get(file_url)

# Преобразуем содержимое в формат JSON
dataset_info = json.loads(response.content)

# Выводим содержимое JSON для анализа
# print(json.dumps(dataset_info, indent=4))
genres = dataset_info['huggan--wikiart']['features']['genre']['names']

artists =  dataset_info['huggan--wikiart']['features']['artist']['names']
artists_dict = {i: author for i, author in enumerate(artists)}

styles =  dataset_info['huggan--wikiart']['features']['style']['names']
styles_dict = {i: s for i, s in enumerate(styles)}

print(artists_dict)
print(styles_dict)
genres_str = '\n'.join([f'{i}: {g}' for i, g in enumerate(genres)])
genres_str

{0: 'Unknown Artist', 1: 'boris-kustodiev', 2: 'camille-pissarro', 3: 'childe-hassam', 4: 'claude-monet', 5: 'edgar-degas', 6: 'eugene-boudin', 7: 'gustave-dore', 8: 'ilya-repin', 9: 'ivan-aivazovsky', 10: 'ivan-shishkin', 11: 'john-singer-sargent', 12: 'marc-chagall', 13: 'martiros-saryan', 14: 'nicholas-roerich', 15: 'pablo-picasso', 16: 'paul-cezanne', 17: 'pierre-auguste-renoir', 18: 'pyotr-konchalovsky', 19: 'raphael-kirchner', 20: 'rembrandt', 21: 'salvador-dali', 22: 'vincent-van-gogh', 23: 'hieronymus-bosch', 24: 'leonardo-da-vinci', 25: 'albrecht-durer', 26: 'edouard-cortes', 27: 'sam-francis', 28: 'juan-gris', 29: 'lucas-cranach-the-elder', 30: 'paul-gauguin', 31: 'konstantin-makovsky', 32: 'egon-schiele', 33: 'thomas-eakins', 34: 'gustave-moreau', 35: 'francisco-goya', 36: 'edvard-munch', 37: 'henri-matisse', 38: 'fra-angelico', 39: 'maxime-maufra', 40: 'jan-matejko', 41: 'mstislav-dobuzhinsky', 42: 'alfred-sisley', 43: 'mary-cassatt', 44: 'gustave-loiseau', 45: 'fernando-bo

'0: abstract_painting\n1: cityscape\n2: genre_painting\n3: illustration\n4: landscape\n5: nude_painting\n6: portrait\n7: religious_painting\n8: sketch_and_study\n9: still_life\n10: Unknown Genre'

In [17]:
prompt_t = """The image is painting by {artist}. The style of painting is {style}. There are 11 possible genres:\n{genres_str}.\nChoose the genre of painting from list. Say only the number of genre, do not output anything else."""
print(prompt_t)

The image is painting by {artist}. The style of painting is {style}. There are 11 possible genres:
{genres_str}.
Choose the genre of painting from list. Say only the number of genre, do not output anything else.


In [3]:
prompt_t = """There are 11 possible genres:\n{genres_str}.\nWhat is the genre of painting by {artist}? Choose from list. Say only the number of genre, do not output anything else."""

In [18]:
prompt_t = """What is the genre of painting image according to the list below:\n{genres_str}\nSay only the number of genre, do not output anything else."""

In [79]:
prompt_t = """The task is to classify an image into one of the following 11 genres:\n{genres_str}.\nYou are provided with the name of the artist and the painting style of the image. Based on this information, determine the correct genre. Choose the genre from the list by providing only the corresponding number. Say **ONLY** the number of the genre and **do not say anything else**.\nArtist: {artist}\nStyle: {style}"""

In [76]:
import time
import re
from PIL import Image
from io import BytesIO


times=[]
token=[]
tok_per_second=[]
predictions = []
correct_predictions = 0  # Счётчик правильных предсказаний
total_predictions = len(df)
true_genres = df['genre'].tolist()

for idx, row in df.iterrows():
    if idx % 100 == 0:
        print(idx)

    image_bytes = row['image']['bytes']
    image_stream = BytesIO(image_bytes)
    image = Image.open(image_stream)
    # print(type(image))
    artist = artists_dict[row['artist']]
    style = styles_dict[row['style']]
    prompt = prompt_t.format(genres_str=genres_str, artist=artist, style=style)

    # Start generation
    start_time = time.time()
    messages = [
        {"role": "system", "content": [
            {"type": "text", "text": """You are an art history and painting styles expert."""}
        ]},
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": prompt}
        ]}
    ]
    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
    # print(input_text)
    inputs = processor(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    # print(inputs.keys())
    # print(inputs['pixel_values'].shape)

    outputs = model.generate(**inputs, max_new_tokens=30)
    # End generation
    end_time = time.time() - start_time
    times.append(end_time)
    # print(f'Time per message: {end_time:.2f} s')

    outputs_text = processor.decode(outputs[0])
    # print(outputs_text)
    regexp_res = re.search(r'<\|start_header_id\|>assistant<\|end_header_id\|>\D*(\d+)', outputs_text)
    if regexp_res:
        answer = regexp_res.group(1)
        if len(answer) > 2:
            answer = '12'
    else:
        answer = '12'
    pred = int(eval(answer))

    count_token = len(tokenizer(outputs_text, return_tensors="pt")['input_ids'][0].to("cpu"))
    # print(f'Token per message: {count_token}')
    token.append(count_token)

    token_per_second = count_token / end_time
    # print(f'Token per second: {token_per_second}')
    tok_per_second.append(token_per_second)
    # Истинный жанр
    true_genre = row['genre']
    print(f'{answer} {true_genre}')
    if pred == true_genre:
        correct_predictions += 1

    # Сохраняем предсказания в массив
    predictions.append(pred)

0
4 4
7 7
6 6
4 2
4 10
9 2
9 10
4 4
7 7
6 6
4 4
4 2
4 4
6 6
6 6
9 9
4 8
7 3
4 4
7 3
6 8
4 4
4 1
4 8
6 6
4 1
4 4
9 10
4 8
5 5
6 6
7 7
9 9
6 10
7 6
4 4
7 7
4 10
5 5
6 6
9 9
6 6
4 8
4 4
9 9
5 2
6 6
4 4
6 8
9 2
9 6
6 2
9 8
9 10
4 4
6 6
4 1
4 4
6 5
4 4
6 6
7 2
7 3
4 10
4 2
4 10
7 3
6 6
7 2
6 7
6 6
4 4
9 2
7 7
6 6
6 6
4 1
4 4
4 4
4 1
4 4
4 2
7 3
4 4
9 10
4 1
4 1
4 2
4 1
5 5
4 2
8 10
4 4
6 6
4 4
4 4
4 2
4 4
6 10
0 9
100
6 2
4 4
9 9
0 10
4 4
4 8
5 5
4 4
0 9
4 2
4 4
6 10
4 4
9 10
6 2
9 2
4 4
4 4
6 8
4 1
4 7
5 3
4 1
4 3
4 4
6 6
4 4
6 8
4 8
4 2
7 3
6 6
4 4
9 2
4 4
6 2
4 4
4 2
4 4
6 3
9 9
4 4
4 8
4 1
4 4
9 10
4 8
6 6
4 2
4 4
4 2
4 4
4 4
4 10
7 3
4 1
9 1
4 10
4 4
4 1
9 10
4 4
4 4
4 4
9 8
7 7
6 6
7 7
4 8
4 2
7 7
4 4
9 8
4 10
4 4
5 5
9 9
5 5
4 2
5 10
4 3
7 8
4 10
4 2
4 2
4 2
5 5
4 10
9 2
10 6
9 2
6 10
9 8
6 10
6 6
5 5
4 4
4 8
4 1
6 6
200
4 4
0 5
9 10
9 4
7 3
7 7
0 8
4 4
4 10
4 8
9 2
4 1
9 2
9 8
6 6
4 8
4 10
4 8
5 2
4 4
4 4
4 10
7 3
9 10
6 10
6 2
6 6
4 4
4 4
5 2
6 6
4 10
7 3
4 1
4 10
10 2
9 8
6 10
4 1

In [51]:
outputs_text

"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n<|image|>The task is to classify a painting into one of the following 11 genres:\n0: abstract_painting\n1: cityscape\n2: genre_painting\n3: illustration\n4: landscape\n5: nude_painting\n6: portrait\n7: religious_painting\n8: sketch_and_study\n9: still_life\n10: Unknown Genre.\nYou are provided with the name of the artist and the painting style. Based on this information, determine the correct genre. Select the most likely genre from the list by providing only the corresponding number. Say only the number of the genre and do not say anything else.\nArtist: pablo-picasso\nStyle: Post_Impressionism<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTo determine the correct genre for Pablo Picasso's work, we need to consider the characteristics of Post-Impressionism and the typical subjects or themes associated"

In [78]:
accuracy = correct_predictions / len(predictions)

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 48.14%


In [65]:
import numpy as np

print(len(times))
print(np.mean(times))
print(np.mean(token))
print(np.mean(tok_per_second))

1132
0.4086772841615306
188.53180212014135
462.1872172534536


In [22]:
df_results = pd.DataFrame({
    'prediction': predictions,
    'true': true_genres
})

# Сохранение DataFrame в CSV файл
df_results.to_csv('./predictions_and_trues.csv')

In [66]:
for i, p in enumerate(predictions):
    if p == 10:
        print(i, true_genres[i])

212 2
456 10
458 2
914 10
1119 6


In [ ]:
prompt_t = """The task is to classify an image into one of the following 11 genres:\n{genres_str}.\nYou are provided with the name of the artist: and the painting style of the image.\nArtist: {artist}\nStyle: {style}\nUse all your knowledge and given information to determine the correct genre. Choose the genre from the list by providing only the corresponding number. Say **ONLY** the number of the genre and **do not say anything else**. THIS IS VEY IMPORTANT FOR MY CAREER I WILL TIP YOU 1000000000$ for the correct answer."""

In [80]:
import time
import re
from PIL import Image
from io import BytesIO


times=[]
token=[]
tok_per_second=[]
predictions = []
correct_predictions = 0  # Счётчик правильных предсказаний
total_predictions = len(df)
true_genres = df['genre'].tolist()

for idx, row in df.iterrows():
    if idx % 100 == 0:
        print(idx)

    image_bytes = row['image']['bytes']
    image_stream = BytesIO(image_bytes)
    image = Image.open(image_stream)
    # print(type(image))
    artist = artists_dict[row['artist']]
    style = styles_dict[row['style']]
    prompt = prompt_t.format(genres_str=genres_str, artist=artist, style=style)

    # Start generation
    start_time = time.time()
    messages = [
        # {"role": "system", "content": [
        #     {"type": "text", "text": """You are an art history and painting styles expert."""}
        # ]},
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": prompt}
        ]}
    ]
    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
    # print(input_text)
    inputs = processor(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    # print(inputs.keys())
    # print(inputs['pixel_values'].shape)

    outputs = model.generate(**inputs, max_new_tokens=30)
    # End generation
    end_time = time.time() - start_time
    times.append(end_time)
    # print(f'Time per message: {end_time:.2f} s')

    outputs_text = processor.decode(outputs[0])
    # print(outputs_text)
    regexp_res = re.search(r'<\|start_header_id\|>assistant<\|end_header_id\|>\D*(\d+)', outputs_text)
    if regexp_res:
        answer = regexp_res.group(1)
        if len(answer) > 2:
            answer = '12'
    else:
        answer = '12'
    pred = int(eval(answer))

    count_token = len(tokenizer(outputs_text, return_tensors="pt")['input_ids'][0].to("cpu"))
    # print(f'Token per message: {count_token}')
    token.append(count_token)

    token_per_second = count_token / end_time
    # print(f'Token per second: {token_per_second}')
    tok_per_second.append(token_per_second)
    # Истинный жанр
    true_genre = row['genre']
    print(f'{answer} {true_genre}')
    if pred == true_genre:
        correct_predictions += 1

    # Сохраняем предсказания в массив
    predictions.append(pred)

0
4 4
7 7
6 6
4 2
4 10
2 2
9 10
4 4
7 7
6 6
4 4
9 2
4 4
6 6
6 6
3 9
6 8
7 3
4 4
7 3
6 8
4 4
4 1
6 8
6 6
4 1
4 4
9 10
4 8
5 5
6 6
7 7
9 9
6 10
7 6
4 4
7 7
9 10
5 5
6 6
9 9
6 6
7 8
4 4
9 9
5 2
6 6
4 4
6 8
6 2
6 6
3 2
3 8
6 10
4 4
6 6
4 1
4 4
6 5
4 4
6 6
7 2
7 3
4 10
4 2
4 10
7 3
6 6
7 2
9 7
6 6
4 4
9 2
7 7
6 6
6 6
4 1
4 4
4 4
4 1
4 4
4 2
7 3
4 4
9 10
4 1
4 1
9 2
6 1
5 5
4 2
9 10
4 4
6 6
4 4
4 4
4 2
4 4
6 10
9 9
100
6 2
4 4
9 9
6 10
4 4
4 8
6 5
4 4
6 9
7 2
4 4
6 10
4 4
9 10
6 2
9 2
4 4
4 4
6 8
4 1
4 7
7 3
4 1
3 3
4 4
6 6
4 4
6 8
4 8
9 2
7 3
6 6
4 4
2 2
4 4
9 2
4 4
9 2
4 4
9 3
9 9
4 4
6 8
4 1
4 4
9 10
4 8
6 6
4 2
4 4
4 2
4 4
4 4
4 10
7 3
4 1
6 1
4 10
4 4
9 1
4 10
4 4
4 4
4 4
4 8
7 7
6 6
7 7
4 8
4 2
7 7
4 4
9 8
7 10
6 4
5 5
9 9
5 5
9 2
6 10
4 3
7 8
4 10
8 2
6 2
4 2
5 5
4 10
6 2
6 6
6 2
8 10
8 8
6 10
6 6
5 5
4 4
4 8
1 1
6 6
200
4 4
5 5
9 10
4 4
7 3
7 7
8 8
4 4
4 10
6 8
8 2
4 1
9 2
9 8
6 6
4 8
4 10
4 8
6 2
4 4
4 4
4 10
6 3
9 10
6 10
8 2
6 6
4 4
4 4
5 2
6 6
6 10
7 3
4 1
3 10
9 2
8 8
6 10
9 10


In [81]:
accuracy = correct_predictions / len(predictions)

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 50.44%


In [6]:
prompt_t = """You are an art expert tasked with identifying the genre of a painting. There are 11 possible genres, which are:\n{genres_str}.\nThe artist is {artist}, and the painting style is {style}. Think step by step to analyze the image. If you are uncertain, choose the genre most commonly associated with the artist and style. Answer **only with the number** corresponding to the genre, and nothing else."""

In [9]:
import time
import re
from PIL import Image
from io import BytesIO


times=[]
# token=[]
# tok_per_second=[]
predictions = []
correct_predictions = 0  # Счётчик правильных предсказаний
total_predictions = len(df)
true_genres = df['genre'].tolist()

for idx, row in df.sample(frac=1).iterrows():
    if idx % 100 == 0:
        print(idx)

    image_bytes = row['image']['bytes']
    image_stream = BytesIO(image_bytes)
    image = Image.open(image_stream)
    # print(type(image))
    artist = artists_dict[row['artist']]
    style = styles_dict[row['style']]
    prompt = prompt_t.format(genres_str=genres_str, artist=artist, style=style)

    # Start generation
    start_time = time.time()
    messages = [
        # {"role": "system", "content": [
        #     {"type": "text", "text": """You are an art expert. You know all genres, styles and artists and carefully adjust your knowledge thinking step by step."""}
        # ]},
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": prompt}
        ]}
    ]
    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
    # print(input_text)
    inputs = processor(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    # print(inputs.keys())
    # print(inputs['pixel_values'].shape)

    outputs = model.generate(**inputs, max_new_tokens=30)
    # End generation
    end_time = time.time() - start_time
    times.append(end_time)
    # print(f'Time per message: {end_time:.2f} s')

    outputs_text = processor.decode(outputs[0])
    print(outputs_text)
    regexp_res = re.search(r'<\|start_header_id\|>assistant<\|end_header_id\|>\D*(\d+)', outputs_text)
    if regexp_res:
        answer = regexp_res.group(1)
        if len(answer) > 2:
            answer = '12'
    else:
        answer = '12'
    pred = int(eval(answer))

    # count_token = len(tokenizer(outputs_text, return_tensors="pt")['input_ids'][0].to("cpu"))
    # print(f'Token per message: {count_token}')
    # token.append(count_token)

    # token_per_second = count_token / end_time
    # print(f'Token per second: {token_per_second}')
    # tok_per_second.append(token_per_second)
    # Истинный жанр
    true_genre = row['genre']
    print(f'{answer} {true_genre}')
    if pred == true_genre:
        correct_predictions += 1

    # Сохраняем предсказания в массив
    predictions.append(pred)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>You are an art expert tasked with identifying the genre of a painting. There are 11 possible genres, which are:
0: abstract_painting
1: cityscape
2: genre_painting
3: illustration
4: landscape
5: nude_painting
6: portrait
7: religious_painting
8: sketch_and_study
9: still_life
10: Unknown Genre.
The artist is claude-monet, and the painting style is Impressionism. Think step by step to analyze the image. If you are uncertain, choose the genre most commonly associated with the artist and style. Answer **only with the number** corresponding to the genre, and nothing else.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

1<|eot_id|>
1 1
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>You are an art expert tasked with identifying the genre of a painting. There are 11 possible genres, which are:
0: abstract_painting
1: cityscape
2: genre_painting
3: illustration
4: landscape
5: nude_painting
6: por

In [10]:
accuracy = correct_predictions / len(predictions)

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 48.23%


: 

In [17]:
torch.cuda.empty_cache()

In [4]:
%pip install pytorch torchvision torchaudio

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  Using cached torchvision-0.19.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.0 kB)
  Using cached torchaudio-2.4.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.4 kB)
  Using cached torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
Using cached torchvision-0.19.1-cp310-cp310-manylinux1_x86_64.whl (7.0 MB)
Using cached torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl (797.1 MB)
Using cached torchaudio-2.4.1-cp310-cp310-manylinux1_x86_64.whl (3.4 MB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-1qvvs5ty/pytorch_7aba008f97484010ad89bd3d3dd203ff/setup.py", line 15, in <module>
          raise Exception(message)
      Ex

In [17]:
%pip install git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830 accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/transformers (to revision 21fac7abba2a37fae86106f87fcf9974fd1e3830) to /tmp/pip-req-build-ntdpz20i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ntdpz20i
  Running command git rev-parse -q --verify 'sha^21fac7abba2a37fae86106f87fcf9974fd1e3830'
  Running command git fetch -q https://github.com/huggingface/transformers 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Running command git checkout -q 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Resolved https://github.com/huggingface/transformers to commit 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 9.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9677030 sha256=3b189c6a8039b44c807a

In [1]:
%pip install -U flash-attn --no-build-isolation

  Using cached flash_attn-2.6.3.tar.gz (2.6 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      
      
      torch.__version__  = 2.4.1+cu121
      
      
      /tmp/pip-install-65vsbgoh/flash-attn_8157ca0660074b44b03d07093d3ef2b8/setup.py:95: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
        warnings.warn(
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-65vsbgoh/flash-attn_8157ca0660074b44b03d07093d3ef2b8/setup.py", line 179, in 

In [1]:
%pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 14.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from PIL import Image
import requests
import torch
from torchvision import io
from typing import Dict
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# Load the model in half-precision on the available device(s)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map=device
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")


ModuleNotFoundError: No module named 'torchvision'

In [1]:
import torch
from torchvision import io
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info

# Load the model in 8-bit precision using BitsAndBytesConfig
bits_and_bytes_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 8-bit precision
    low_cpu_mem_usage=True  # Reduce CPU memory usage
)

# Detect device (CUDA or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model with 8-bit precision
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    quantization_config=bits_and_bytes_config,  # Apply the 8-bit config
    device_map="auto"  # Automatically map to available devices
)

# Load the processor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")


Unused kwargs: ['low_cpu_mem_usage']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [2]:
import pandas as pd
import requests
import json

df = pd.read_parquet('./train-00000-of-00072.parquet')

file_url = 'https://huggingface.co/datasets/huggan/wikiart/resolve/main/dataset_infos.json'
response = requests.get(file_url)

# Преобразуем содержимое в формат JSON
dataset_info = json.loads(response.content)

# Выводим содержимое JSON для анализа
# print(json.dumps(dataset_info, indent=4))
genres = dataset_info['huggan--wikiart']['features']['genre']['names']

artists =  dataset_info['huggan--wikiart']['features']['artist']['names']
artists_dict = {i: author for i, author in enumerate(artists)}

styles =  dataset_info['huggan--wikiart']['features']['style']['names']
styles_dict = {i: s for i, s in enumerate(styles)}

print(artists_dict)
print(styles_dict)
genres_str = '\n'.join([f'{i}: {g}' for i, g in enumerate(genres)])
genres_str

{0: 'Unknown Artist', 1: 'boris-kustodiev', 2: 'camille-pissarro', 3: 'childe-hassam', 4: 'claude-monet', 5: 'edgar-degas', 6: 'eugene-boudin', 7: 'gustave-dore', 8: 'ilya-repin', 9: 'ivan-aivazovsky', 10: 'ivan-shishkin', 11: 'john-singer-sargent', 12: 'marc-chagall', 13: 'martiros-saryan', 14: 'nicholas-roerich', 15: 'pablo-picasso', 16: 'paul-cezanne', 17: 'pierre-auguste-renoir', 18: 'pyotr-konchalovsky', 19: 'raphael-kirchner', 20: 'rembrandt', 21: 'salvador-dali', 22: 'vincent-van-gogh', 23: 'hieronymus-bosch', 24: 'leonardo-da-vinci', 25: 'albrecht-durer', 26: 'edouard-cortes', 27: 'sam-francis', 28: 'juan-gris', 29: 'lucas-cranach-the-elder', 30: 'paul-gauguin', 31: 'konstantin-makovsky', 32: 'egon-schiele', 33: 'thomas-eakins', 34: 'gustave-moreau', 35: 'francisco-goya', 36: 'edvard-munch', 37: 'henri-matisse', 38: 'fra-angelico', 39: 'maxime-maufra', 40: 'jan-matejko', 41: 'mstislav-dobuzhinsky', 42: 'alfred-sisley', 43: 'mary-cassatt', 44: 'gustave-loiseau', 45: 'fernando-bo

'0: abstract_painting\n1: cityscape\n2: genre_painting\n3: illustration\n4: landscape\n5: nude_painting\n6: portrait\n7: religious_painting\n8: sketch_and_study\n9: still_life\n10: Unknown Genre'

In [3]:
import base64
from io import BytesIO
from PIL import Image

# Convert the image bytes to base64
def image_to_base64(image):
    
    buffered = BytesIO()
    image.save(buffered, format="PNG")  # You can change the format to match the image type
    base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
    base64_image = f"data:image/png;base64,{base64_image}"
    return base64_image


In [43]:
from PIL import Image
from io import BytesIO

for i, row in df.iterrows():
    image_bytes = row['image']['bytes']
    image_stream = BytesIO(image_bytes)
    image = Image.open(image_stream)
    img_file = BytesIO()
    image.save(img_file, 'png', quality='keep')
    if img_file.tell() / 1024 <= 1200:
        print("-"*10)
        print(i)
        print(image.size)
        print(img_file.tell() / 1024)
        print("-"*10)
    


----------
0
(750, 597)
880.994140625
----------
----------
105
(2196, 1381)
1089.7509765625
----------
----------
225
(1382, 2230)
884.537109375
----------
----------
333
(1382, 2764)
1195.640625
----------
----------
417
(1382, 1881)
779.4794921875
----------
----------
476
(1381, 1916)
789.5537109375
----------
----------
491
(1908, 1382)
1155.20703125
----------
----------
569
(1382, 2056)
1011.634765625
----------
----------
898
(1382, 1629)
973.818359375
----------
----------
948
(1732, 1382)
629.2412109375
----------
----------
976
(1382, 1756)
1105.646484375
----------
----------
979
(2069, 1382)
884.9697265625
----------


In [4]:
prompt_t = """There are 11 possible genres:\n{genres_str}.\nWhat is the genre of painting by {artist}? Choose from list. Say only the number of genre, do not output anything else."""

In [8]:
image.save('./img_0.jpg', optimize=True, quality=50)

In [9]:
torch.cuda.empty_cache()

In [20]:
image_bytes = df.loc[0]['image']['bytes']
image_stream = BytesIO(image_bytes)
image = Image.open(image_stream)

image.size

(750, 597)

In [15]:
torch.cuda.empty_cache()

In [16]:
import time
import re
from PIL import Image
from io import BytesIO


times=[]
# token=[]
# tok_per_second=[]
predictions = []
correct_predictions = 0
total_predictions = len(df)
true_genres = df['genre'].tolist()

for idx, row in df.iterrows():
   
    if idx % 100 == 0:
        print(idx)

    image_bytes = row['image']['bytes']
    image_stream = BytesIO(image_bytes)
    image = Image.open(image_stream)
    img_file = BytesIO()
    image.save(img_file, 'jpeg')
    # img_file_size = img_file.tell() / 1024
    # print(img_file_size)
    # coef = int(880.99 / img_file_size * 100)
    # print(coef)
    # image.resize((224, 224))
    del img_file
    img_file = BytesIO()
    image.save(img_file, 'jpeg', quality=10)
    image = Image.open(img_file)
    print(img_file.tell() / 1024)
    del img_file

    artist = artists_dict[row['artist']]
    style = styles_dict[row['style']]
    prompt = prompt_t.format(genres_str=genres_str, artist=artist, style=style)
    
    image_base64_str = image_to_base64(image)
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_base64_str},  # Base64 encoded image with MIME type
                {"type": "text", "text": prompt},
            ],
        }
    ]

    # Start generation
    start_time = time.time()
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, _ = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(device)

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=10)
    generated_ids_trimmed = [
        out_ids[len(in_ids):].cpu() for in_ids, out_ids in zip(inputs.input_ids.to("cpu"), generated_ids.to("cpu"))
    ]

    # Декодируем на CPU
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    print(output_text)
    
    # End generation
    end_time = time.time() - start_time
    times.append(end_time)
    print(f'Time per message: {end_time:.2f} s')

    # regexp_res = re.search(r'<\|start_header_id\|>assistant<\|end_header_id\|>\D*(\d+)', output_text)
    # if regexp_res:
    #     answer = regexp_res.group(1)
    #     if len(answer) > 2:
    #         answer = '12'
    # else:
    #     answer = '12'
    pred = int(eval(output_text[0]))

    # count_token = len(tokenizer(outputs_text, return_tensors="pt")['input_ids'][0].to("cpu"))
    # print(f'Token per message: {count_token}')
    # token.append(count_token)

    # token_per_second = count_token / end_time
    # print(f'Token per second: {token_per_second}')
    # tok_per_second.append(token_per_second)
    
    # Истинный жанр
    true_genre = row['genre']
    if pred == true_genre:
        correct_predictions += 1

    # Сохраняем предсказания в массив
    predictions.append(pred)
    del inputs
    del image_inputs
    # del video_inputs
    del generated_ids
    del generated_ids_trimmed
    torch.cuda.empty_cache()

0
0.6083984375
['4']
Time per message: 0.27 s
0.6083984375


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.99 GiB. GPU 0 has a total capacity of 23.64 GiB of which 1.47 GiB is free. Including non-PyTorch memory, this process has 22.17 GiB memory in use. Of the allocated memory 21.42 GiB is allocated by PyTorch, and 296.85 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [13]:
image.save('omg_0.jpg')